Note: I left the code to generate the htfs for each test system in the notebook, but if you don't want to re-generate, you can use the pickled htf that I saved (I left the full path in for you). I generated these htfs based after you added support for `lambda_{i}_bonds_old/new` and `scale_lambda_0_bonds`

In [1]:
# from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
# from simtk import unit, openmm
# from perses.tests.utils import compute_potential_components
# from openmmtools.constants import kB
# from perses.dispersed.utils import configure_platform
# from perses.annihilation.rest import RESTTopologyFactory
# from perses.annihilation.lambda_protocol import RESTState
# import numpy as np
# from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
# import itertools
from perses.tests.test_topology_proposal import generate_dipeptide_top_pos_sys, generate_atp
import pickle
# import copy

# #############################################
# # CONSTANTS
# #############################################
# temperature = 298.0 * unit.kelvin
# kT = kB * temperature
# beta = 1.0/kT
# REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CUDA")


INFO:rdkit:Enabling RDKit 2021.03.1 jupyter extensions


In [2]:
def test_torsion_energies(htf, is_old=True, is_solvated=False, check_scale=False):
    htf_copy = copy.deepcopy(htf)
    
    # Get periodic torsion force and old/new positions
    system = htf._topology_proposal.old_system if is_old else htf._topology_proposal.new_system
    periodic_torsion_force = system.getForce(2) 
    positions = htf.old_positions(htf.hybrid_positions) if is_old else htf.new_positions(htf.hybrid_positions)
    
    # Get custom torsion force and hybrid positions
    torsion_force_index = 3 if is_solvated else 2
    hybrid_system = htf.hybrid_system
    custom_torsion_force = hybrid_system.getForce(torsion_force_index)
    hybrid_positions = htf.hybrid_positions
    
    # Set global parameters
    lambda_old = 1 if is_old else 0
    lambda_new = 0 if is_old else 1
    for i in range(custom_torsion_force.getNumGlobalParameters()):
        if custom_torsion_force.getGlobalParameterName(i) == 'lambda_0_torsions_old':
            custom_torsion_force.setGlobalParameterDefaultValue(i, lambda_old)
        elif custom_torsion_force.getGlobalParameterName(i) == 'lambda_0_torsions_new':
            custom_torsion_force.setGlobalParameterDefaultValue(i, lambda_new)

    # Zero the unique old/new torsions in the custom torsion force
    hybrid_to_torsion_indices = htf._hybrid_to_new_torsion_indices if is_old else htf._hybrid_to_old_torsion_indices
    print(hybrid_to_torsion_indices)
    for hybrid_idx, idx in hybrid_to_torsion_indices.items():
        p1, p2, p3, p4, hybrid_params = custom_torsion_force.getTorsionParameters(hybrid_idx)
        hybrid_params = list(hybrid_params)
#         index_to_zero = -4 if is_old else -1
#         hybrid_params[index_to_zero] *= 0
        hybrid_params[-4] *= 0
        hybrid_params[-1] *= 0
        custom_torsion_force.setTorsionParameters(hybrid_idx, p1, p2, p3, p4, hybrid_params)
        
    ## Get energy components of standard torsion force
    platform = configure_platform(REFERENCE_PLATFORM)
    thermostate_other = ThermodynamicState(system=system, temperature=temperature)
    integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
    context_other = thermostate_other.create_context(integrator_other)
    context_other.setPositions(positions)
    components_other = compute_potential_components(context_other, beta=beta)
    print(components_other)
    
    ## Get energy components of custom torsion force
    if is_solvated:
        custom_torsion_force.setUsesPeriodicBoundaryConditions(True)
    thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
    integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
    context_hybrid.setPositions(hybrid_positions)
    components_hybrid = compute_potential_components(context_hybrid, beta=beta)
    print(components_hybrid)
    
    assert np.isclose([components_other[2][1]], [components_hybrid[2][1]])
    
    print("Success! Custom torsion force and standard torsion force energies are equal!")
    
    if check_scale:
        
        ## Get custom torsion force and hybrid positions
        torsion_force_index = 3 if is_solvated else 2
        hybrid_system = htf_copy.hybrid_system
        custom_torsion_force = hybrid_system.getForce(torsion_force_index)
        hybrid_positions = htf_copy.hybrid_positions
        
        if is_solvated:
            custom_torsion_force.setUsesPeriodicBoundaryConditions(True)
        
        ## Get energy components of custom torsion force
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid = compute_potential_components(context_hybrid, beta=beta)
        print(components_hybrid)
        
        # Set `scale_lambda_{i}` to 0.5
        for i in range(custom_torsion_force.getNumGlobalParameters()):
            if custom_torsion_force.getGlobalParameterName(i) == 'scale_lambda_0_torsions':
                custom_torsion_force.setGlobalParameterDefaultValue(i, 0.5)
            elif custom_torsion_force.getGlobalParameterName(i) == 'interscale_lambda_0_torsions':
                custom_torsion_force.setGlobalParameterDefaultValue(i, 0.5)
        
        ## Get energy components of custom torsion force iwth scaling
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid_scaled = compute_potential_components(context_hybrid, beta=beta)
        print(components_hybrid_scaled)
        
        assert not np.isclose([components_hybrid[2][1]], [components_hybrid_scaled[2][1]])
        
        print("Success! Scaling the torsion force changes the energy")

# Alanine dipeptide in vacuum

In [6]:
# Create a htf
atp, system_generator = generate_atp(phase = 'vacuum')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 8 new atoms
INFO:geometry:	reduced angle potential = 0.02272230759221436.
INFO:geometry:	reduced angle potential = 0.4679350024115357.
INFO:geometry:	reduced angle potential = 0.040774130825061816.
INFO:geometry:	reduced angle potential = 0.3397371723547433.
INFO:geometry:	reduced angle potential = 0.515204993442505.
INFO:geometry:	reduced angle potential = 0.764116245509211.
INFO:geometry:	reduced angle potential = 0.1330557289580421.
INFO:geometry:	reduced angle potential = 0.07624795173393698.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angle forc

...

INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986687678176
INFO:geometry:final reduced energy 29.78946596595154
INFO:geometry:sum of energies: 29.789467829390215
INFO:geometry:magnitude of difference in the energies: 1.8634386762528266e-06
INFO:geometry:Final logp_proposal: -27010.885813757428


added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463344085812), ('CustomBondForce', 12.970883364040223)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda'], ['nonscale_region']]
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO

In [7]:
# Save htf
with open("atp_vacuum.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [21]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [24]:
test_torsion_energies(htf)

{50: 8, 51: 9, 52: 10, 58: 16, 59: 17, 60: 18, 61: 19, 62: 20, 63: 21, 64: 22, 65: 23, 66: 24, 70: 28, 71: 29, 72: 30, 73: 31, 74: 32, 75: 33, 76: 34, 79: 37, 80: 38, 81: 39, 84: 42, 85: 43, 86: 44, 87: 45, 88: 46, 89: 47, 90: 48, 91: 49, 92: 50, 93: 51, 94: 52, 95: 53, 99: 57, 100: 58, 101: 59, 102: 60, 103: 61, 104: 62, 105: 63, 106: 64, 107: 65, 108: 66, 109: 67}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', -39.4499277456003), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.691138178270751), ('CustomAngleForce', 13.805947690407493), ('CustomTorsionForce', 16.28540329352652), ('AndersenThermostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [25]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [26]:
test_torsion_energies(htf, is_old=False)

{8: 8, 9: 9, 10: 10, 16: 16, 17: 17, 18: 18, 24: 24, 25: 25, 26: 26, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.6906144830418987), ('HarmonicAngleForce', 13.351800265281843), ('PeriodicTorsionForce', 26.41852205265097), ('NonbondedForce', -54.709291264726836), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.691138178270751), ('CustomAngleForce', 13.805947690407493), ('CustomTorsionForce', 26.418526844667138), ('AndersenThermostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


### Test one alchemical region, one scale region

In [27]:
# Create a htf
atp, system_generator = generate_atp(phase = 'vacuum')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True, 
                                scale_regions=[[10, 11, 12, 13, 22, 23, 24, 25, 26, 27, 28, 29]])

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 8 new atoms
INFO:geometry:	reduced angle potential = 0.20327771654873789.
INFO:geometry:	reduced angle potential = 0.511420880544537.
INFO:geometry:	reduced angle potential = 0.04764227941841384.
INFO:geometry:	reduced angle potential = 0.37038219887993884.
INFO:geometry:	reduced angle potential = 0.3247431386698406.
INFO:geometry:	reduced angle potential = 0.048769497076544176.
INFO:geometry:	reduced angle potential = 0.4290259864776433.
INFO:geometry:	reduced angle potential = 0.6637207074188998.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angle fo

...

INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986687678176
INFO:geometry:final reduced energy 29.78946596595154
INFO:geometry:sum of energies: 29.789467829390215
INFO:geometry:magnitude of difference in the energies: 1.8634386762528266e-06
INFO:geometry:Final logp_proposal: -26846.123688227624


added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463344085812), ('CustomBondForce', 12.970883364040223)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda', 'scale_lambda_0', 'interscale_lambda_0'], ['nonscale_region', 'scale_region_0', 'interscale_region_0']]
INFO:relative:No MonteCarloBarostat added.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0)

In [28]:
# Save htf
with open("atp_vacuum_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [29]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [30]:
htf.hybrid_system.getForces()

[<simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b6387f54ab0> >,
 <simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2b6387f54db0> >,
 <simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2b6387f54300> >]

In [31]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[10, 11, 12, 13, 22, 23, 24, 25, 26, 27, 28, 29]

In [33]:
test_torsion_energies(htf)

{50: 8, 51: 9, 52: 10, 58: 16, 59: 17, 60: 18, 61: 19, 62: 20, 63: 21, 64: 22, 65: 23, 66: 24, 70: 28, 71: 29, 72: 30, 73: 31, 74: 32, 75: 33, 76: 34, 79: 37, 80: 38, 81: 39, 84: 42, 85: 43, 86: 44, 87: 45, 88: 46, 89: 47, 90: 48, 91: 49, 92: 50, 93: 51, 94: 52, 95: 53, 99: 57, 100: 58, 101: 59, 102: 60, 103: 61, 104: 62, 105: 63, 106: 64, 107: 65, 108: 66, 109: 67}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', -39.4499277456003), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.9115510396450555), ('CustomAngleForce', 7.221285964110747), ('CustomTorsionForce', 16.28540329352652), ('AndersenThermostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [34]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [36]:
test_torsion_energies(htf, is_old=False)

{8: 8, 9: 9, 10: 10, 16: 16, 17: 17, 18: 18, 24: 24, 25: 25, 26: 26, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.911027344416205), ('HarmonicAngleForce', 6.767138538985095), ('PeriodicTorsionForce', 28.901175254389177), ('NonbondedForce', 126.41930931777544), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.9115510396450555), ('CustomAngleForce', 7.221285964110747), ('CustomTorsionForce', 28.90118005134022), ('AndersenThermostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


# Alanine dipeptide in solvent

In [2]:
# Create a htf
atp, system_generator = generate_atp(phase = 'solvent')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [10, 14, 18, 15, 19, 17, 13, 16]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...


generating geometry engine
making geometry proposal from ALA to THR


INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching function, switching distance...
INFO:geometry:		creating nonbonded exception force (i.e. custom bond for 1,4s)...
INFO:geometry:		looping through exceptions calculating growth indices, and adding appropriate interactions to custom bond force.
INFO:geometry:		there are 1654 in the reference Nonbonded force
INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (7, 6, 8, 10, [1, Quantity(value=0.99000

...

( nonscale_lambda_bonds * nonscale_region ) * (K/2)*(r-length)^2;K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;
4
4
elec  [1, 0, 1, 0, 0, 1, Quantity(value=0.1123, unit=elementary charge), Quantity(value=0.1123, unit=elementary charge)]
elec  [1, 0, 1, 0, 0, 1, Quantity(value=-0.3662, unit=elementary charge), Quantity(value=-0.3662, unit=elementary charge)]
elec  [1, 0, 1, 0, 0, 1, Quantity(value=0.1123, unit=elementary charge), Quantity(value=0.1123, unit=elementary charge)]
elec  [1, 0, 1, 0, 0, 1, Quantity(value=0.1123, unit=elementary charge), Quantity(value=0.1123, unit=elementary charge)]
elec  [1, 0, 1, 0, 0, 1, Quantity(value=0.5972, unit=elementary charge), Quantity(value=0.5972, unit=elementary charge)]
elec

elec  [1, 1, 0, 0, 0, 0, Quantity(value=-0.834, unit=elementary charge), Quantity(value=-0.834, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=-0.834, unit=elementary charge), Quantity(value=-0.834, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=-0.834, unit=elementary charge), Quantity(value=-0.834, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.

elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=-0.834, unit=elementary charge), Quantity(value=-0.834, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=-0.834, unit=elementary charge), Quantity(value=-0.834, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=-0.8

elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=-0.834, unit=elementary charge), Quantity(value=-0.834, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=-0.834, unit=elementary charge), Quantity(value=-0.834, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=-0.834, unit=elementary charge), Quantity(value=-0.834, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.

elec  [1, 1, 0, 0, 0, 0, Quantity(value=-0.834, unit=elementary charge), Quantity(value=-0.834, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=-0.834, unit=elementary charge), Quantity(value=-0.834, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=-0.834, unit=elementary charge), Quantity(value=-0.834, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.417, unit=elementary charge), Quantity(value=0.417, unit=elementary charge)]
elec  [1, 1, 0, 0, 0, 0, Quantity(value=0.

In [3]:
# Save htf
with open("atp_solvent.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [4]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

OpenMMException: Unknown property 'param9' in node 'Particle'

In [40]:
test_torsion_energies(htf, is_solvated=True)

{50: 8, 51: 9, 52: 10, 58: 16, 59: 17, 60: 18, 61: 19, 62: 20, 63: 21, 64: 22, 65: 23, 66: 24, 70: 28, 71: 29, 72: 30, 73: 31, 74: 32, 75: 33, 76: 34, 79: 37, 80: 38, 81: 39, 84: 42, 85: 43, 86: 44, 87: 45, 88: 46, 89: 47, 90: 48, 91: 49, 92: 50, 93: 51, 94: 52, 95: 53, 99: 57, 100: 58, 101: 59, 102: 60, 103: 61, 104: 62, 105: 63, 106: 64, 107: 65, 108: 66, 109: 67}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.034783653565214685), ('HarmonicAngleForce', 0.6112775462129015), ('PeriodicTorsionForce', 16.285398289983082), ('NonbondedForce', -953.8255387223031), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.1906714634379987), ('CustomAngleForce', 9.730501694945348), ('CustomTorsionForce', 16.28540329352652), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force a

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [41]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [42]:
test_torsion_energies(htf, is_old=False, is_solvated=True)

{8: 8, 9: 9, 10: 10, 16: 16, 17: 17, 18: 18, 24: 24, 25: 25, 26: 26, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 1.1901477682091637), ('HarmonicAngleForce', 9.276354269819718), ('PeriodicTorsionForce', 30.883917803886508), ('NonbondedForce', -905.3356653618777), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.1906714634379987), ('CustomAngleForce', 9.730501694945348), ('CustomTorsionForce', 30.883922613443637), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


### Test one alchemical region, one scale region

In [43]:
# Create a htf
atp, system_generator = generate_atp(phase = 'solvent')
htf = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                                rxn_field=True,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False,
                                conduct_htf_prop=True, 
                                scale_regions=[[10, 11, 12, 13, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564]])

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{9: 7}, {10: 7}, {11: 7}, {12: 7}, {13: 7}, {9: 8}, {10: 8}, {11: 8}, {12: 8}, {13: 8}, {9: 9}, {10: 9}, {11: 9}, {12: 9}, {13: 9}]
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{10: 7}
INFO:proposal_generator:{11: 7}
INFO:proposal_generator:{12: 7}
INFO:proposal_generator:{13: 7}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:Returning map that best satisfies matching_criterion
IN

making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [10, 18, 14, 13, 17, 19, 16, 15]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching fun

generating geometry engine
making geometry proposal from ALA to THR


INFO:geometry:Neglected angle terms : []
INFO:geometry:omitted_growth_terms: {'bonds': [], 'angles': [], 'torsions': [], '1,4s': []}
INFO:geometry:extra torsions: {0: (7, 6, 8, 10, [1, Quantity(value=0.9900002084555819, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 1]), 1: (19, 18, 10, 8, [1, Quantity(value=-0.07858019216845058, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 6]), 2: (19, 18, 10, 14, [1, Quantity(value=2.118586571250296, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 6]), 3: (19, 18, 10, 13, [1, Quantity(value=-2.149781203448759, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 6]), 4: (8, 10, 14, 17, [1, Quantity(value=-2.1249532223845904, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 5]), 5: (8, 10, 14, 16, [1, Quantity(value=2.0543043017157916, unit=radian), Quantity(value=1200.0, unit=kilocalorie/mole), 7]), 6: (8, 10, 14, 15, [1, Quantity(value=-0.02223392579985184, unit=radian), Quantity(value=1

...

INFO:geometry:total reduced potential before atom placement: 9.11648114171204


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 20.672986687678176
INFO:geometry:final reduced energy 29.78946596595156
INFO:geometry:sum of energies: 29.789467829390215
INFO:geometry:magnitude of difference in the energies: 1.8634386549365445e-06
INFO:geometry:Final logp_proposal: -26846.184056050522


added energy components: [('CustomBondForce', 0.000520203927326505), ('CustomAngleForce', 0.4511197756248138), ('CustomTorsionForce', 7.250463344085812), ('CustomBondForce', 12.970883364040223)]


INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_lambda', 'scale_lambda_0', 'interscale_lambda_0'], ['nonscale_region', 'scale_region_0', 'interscale_region_0']]
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.56477354, y=0.0, z=0.0), unit=nano

In [44]:
# Save htf
with open("atp_solvent_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [45]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [46]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[10, 11, 12, 13, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564]

In [48]:
test_torsion_energies(htf, is_solvated=True)

{50: 8, 51: 9, 52: 10, 58: 16, 59: 17, 60: 18, 61: 19, 62: 20, 63: 21, 64: 22, 65: 23, 66: 24, 70: 28, 71: 29, 72: 30, 73: 31, 74: 32, 75: 33, 76: 34, 79: 37, 80: 38, 81: 39, 84: 42, 85: 43, 86: 44, 87: 45, 88: 46, 89: 47, 90: 48, 91: 49, 92: 50, 93: 51, 94: 52, 95: 53, 99: 57, 100: 58, 101: 59, 102: 60, 103: 61, 104: 62, 105: 63, 106: 64, 107: 65, 108: 66, 109: 67}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.034783653565214685), ('HarmonicAngleForce', 0.6112775462129015), ('PeriodicTorsionForce', 16.285398289983082), ('NonbondedForce', -826.1582702128602), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.6748159227740699), ('CustomAngleForce', 3.8721305328379074), ('CustomTorsionForce', 16.28540329352652), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force 

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [49]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [51]:
test_torsion_energies(htf, is_old=False, is_solvated=True, check_scale=True)

{8: 8, 9: 9, 10: 10, 16: 16, 17: 17, 18: 18, 24: 24, 25: 25, 26: 26, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.6742922275452361), ('HarmonicAngleForce', 3.417983107712269), ('PeriodicTorsionForce', 29.127343616014617), ('NonbondedForce', -807.0147196057264), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.6748159227740699), ('CustomAngleForce', 3.8721305328379074), ('CustomTorsionForce', 29.12734805617243), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.6748159227740699), ('CustomAngleForce', 3.8721305328379074), ('CustomTorsionForce', 16.28540329352652), ('AndersenThermostat

# CDK2 transformation in solvent

In [52]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


In [54]:
# Create a htf
setup_directory = resource_filename("perses", "data/cdk2-example")

# Get options
from perses.app.setup_relative_calculation import getSetupOptions
yaml_filename = os.path.join(setup_directory, "cdk2_setup_repex.yaml")
setup_options = getSetupOptions(yaml_filename)

# Update options
for parameter in ['protein_pdb', 'ligand_file']:
    setup_options[parameter] = os.path.join(setup_directory, setup_options[parameter])
    
setup_options['rxn_field'] = True
setup_options['phases'] = ['solvent']
setup_options['scale_regions'] = None
setup_options['validate_endstate_energies'] = False
    
setup_dict = setup_relative_calculation.run_setup(setup_options, serialize_systems=False, build_samplers=False)

2021-05-05 14:00:46,278:(13.70s):root:	Detecting phases...
2021-05-05 14:00:46,281:(0.00s):root:		phases detected: ['complex', 'solvent', 'vacuum']
2021-05-05 14:00:46,285:(0.00s):root:No constraints will be removed
2021-05-05 14:00:46,287:(0.00s):root:No spectators
2021-05-05 14:00:46,290:(0.00s):root:			run_type is not specified; default to None
2021-05-05 14:00:46,292:(0.00s):root:	Detecting fe_type...
2021-05-05 14:00:46,294:(0.00s):root:		fe_type: repex
2021-05-05 14:00:46,296:(0.00s):root:			offline-freq not specified: default to 10.
2021-05-05 14:00:46,299:(0.00s):root:	'neglect_angles' detected: False.
2021-05-05 14:00:46,300:(0.00s):root:	'softcore_v2' not specified: default to 'False'
2021-05-05 14:00:46,301:(0.00s):root:	Creating 'output'...
2021-05-05 14:00:46,313:(0.01s):root:	detected ligand file: /home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf
2021-05-05 14:00:46,314:(0.00s):r

2021-05-05 14:00:52,426:(0.00s):geometry:Conducting forward proposal...
2021-05-05 14:00:52,429:(0.00s):geometry:Computing proposal order with NetworkX...
2021-05-05 14:00:52,447:(0.02s):geometry:number of atoms to be placed: 1
2021-05-05 14:00:52,450:(0.00s):geometry:Atom index proposal order is [2190]
2021-05-05 14:00:52,452:(0.00s):geometry:omitted_bonds: []
2021-05-05 14:00:52,455:(0.00s):geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
2021-05-05 14:00:53,082:(0.63s):geometry:creating growth system...
2021-05-05 14:00:53,171:(0.09s):geometry:	creating bond force...
2021-05-05 14:00:53,175:(0.00s):geometry:	there are 27 bonds in reference force.
2021-05-05 14:00:53,178:(0.00s):geometry:	creating angle force...
2021-05-05 14:00:53,181:(0.00s):geometry:	there are 85 angles in reference force.
2021-05-05 14:00:53,184:(0.00s):geometry:	creating torsion force...
2021-05-05 14:00:53,187:(0.00s):geometry:	creating extra

conducting subsequent work with the following platform: CUDA


2021-05-05 14:00:55,716:(2.40s):geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


...

2021-05-05 14:01:07,791:(1.77s):geometry:total reduced potential before atom placement: 48.41899607369698


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-05-05 14:01:09,611:(1.82s):geometry:total reduced energy added from growth system: -5.056836675687361
2021-05-05 14:01:09,613:(0.00s):geometry:final reduced energy 43.36216011253515
2021-05-05 14:01:09,615:(0.00s):geometry:sum of energies: 43.362159398009624
2021-05-05 14:01:09,617:(0.00s):geometry:magnitude of difference in the energies: 7.145255302987152e-07
2021-05-05 14:01:09,619:(0.00s):geometry:Final logp_proposal: 11.565896514368891
2021-05-05 14:01:09,772:(0.15s):root:	writing pickle output...


added energy components: [('CustomBondForce', 0.0879842954280575), ('CustomAngleForce', 0.04605409749261916), ('CustomTorsionForce', 0.005423562569706604), ('CustomBondForce', -5.1962986311777435)]


2021-05-05 14:01:09,935:(0.16s):root:	successfully dumped pickle.
2021-05-05 14:01:09,939:(0.00s):root:	setup is complete.  Writing proposals and positions for each phase to top_prop dict...
2021-05-05 14:01:09,941:(0.00s):root:	writing atom_mapping.png
2021-05-05 14:01:10,554:(0.61s):root:	 steps per move application: 50
2021-05-05 14:01:10,557:(0.00s):root:	trajectory prefix: out
2021-05-05 14:01:10,559:(0.00s):root:	atom selection detected: not water
2021-05-05 14:01:10,561:(0.00s):root:	no nonequilibrium detected.
2021-05-05 14:01:10,562:(0.00s):root:	cataloging HybridTopologyFactories...
2021-05-05 14:01:10,564:(0.00s):root:		phase: solvent:
2021-05-05 14:01:10,565:(0.00s):root:		writing HybridTopologyFactory for phase solvent...
2021-05-05 14:01:10,567:(0.00s):relative:*** Generating RxnHybridTopologyFactory ***
2021-05-05 14:01:10,569:(0.00s):relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
2021-05-05 14:01:10,623:(0.05s):relative:Old sy

In [55]:
setup_options

{'protein_pdb': '/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_fixed_nohet.pdb',
 'ligand_file': '/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf',
 'old_ligand_index': 14,
 'new_ligand_index': 15,
 'forcefield_files': ['amber/protein.ff14SB.xml',
  'amber/tip3p_standard.xml',
  'amber/tip3p_HFE_multivalent.xml'],
 'small_molecule_forcefield': 'gaff-2.11',
 'small_molecule_parameters_cache': 'cdk2-cache.json',
 'pressure': 1.0,
 'temperature': 300.0,
 'solvent_padding': 9.0,
 'save_setup_pickle_as': 'fesetup_hbonds.pkl',
 'n_steps_per_move_application': 50,
 'fe_type': 'repex',
 'checkpoint_interval': 250,
 'n_cycles': 2,
 'n_states': 6,
 'n_equilibration_iterations': 0,
 'trajectory_directory': 'output',
 'trajectory_prefix': 'out',
 'atom_selection': 'not water',
 'phases': ['solvent'],
 'protocol-type': 

In [56]:
htf = setup_dict['hybrid_topology_factories']['solvent']

In [57]:
# Save htf
with open("cdk2_solvent.pickle", "wb") as f:
    pickle.dump(htf, f)

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [58]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [60]:
test_torsion_energies(htf, is_solvated=True)

{235: 78, 270: 113, 297: 140, 298: 141, 303: 146}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954756), ('HarmonicAngleForce', 29.199523609209102), ('PeriodicTorsionForce', 15.076747992617829), ('NonbondedForce', 1043.4735475113737), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 34.0975454150916), ('CustomTorsionForce', 15.07675469305873), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [61]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [62]:
test_torsion_energies(htf, is_old=False, is_solvated=True)

{70: 70, 112: 112, 142: 142, 143: 143, 148: 148}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899664), ('HarmonicAngleForce', 34.05118222969635), ('PeriodicTorsionForce', 15.109114462408588), ('NonbondedForce', 1058.4544025553776), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 34.0975454150916), ('CustomTorsionForce', 15.109123120341874), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


### Test one alchemical region, one scale region

In [63]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle

In [64]:
# Create a htf
setup_directory = resource_filename("perses", "data/cdk2-example")

# Get options
from perses.app.setup_relative_calculation import getSetupOptions
yaml_filename = os.path.join(setup_directory, "cdk2_setup_repex.yaml")
setup_options = getSetupOptions(yaml_filename)

# Update options
for parameter in ['protein_pdb', 'ligand_file']:
    setup_options[parameter] = os.path.join(setup_directory, setup_options[parameter])
    
setup_options['rxn_field'] = True
setup_options['phases'] = ['solvent']
setup_options['scale_regions'] = [[44, 2197]]
setup_options['validate_endstate_energies'] = False
    
setup_dict = setup_relative_calculation.run_setup(setup_options, serialize_systems=False, build_samplers=False)

2021-05-05 14:01:40,667:(29.86s):root:	Detecting phases...
2021-05-05 14:01:40,671:(0.00s):root:		phases detected: ['complex', 'solvent', 'vacuum']
2021-05-05 14:01:40,675:(0.00s):root:No constraints will be removed
2021-05-05 14:01:40,677:(0.00s):root:No spectators
2021-05-05 14:01:40,680:(0.00s):root:			run_type is not specified; default to None
2021-05-05 14:01:40,682:(0.00s):root:	Detecting fe_type...
2021-05-05 14:01:40,684:(0.00s):root:		fe_type: repex
2021-05-05 14:01:40,686:(0.00s):root:			offline-freq not specified: default to 10.
2021-05-05 14:01:40,687:(0.00s):root:	'neglect_angles' detected: False.
2021-05-05 14:01:40,688:(0.00s):root:	'softcore_v2' not specified: default to 'False'
2021-05-05 14:01:40,689:(0.00s):root:	Creating 'output'...
2021-05-05 14:01:40,701:(0.01s):root:	detected ligand file: /home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf
2021-05-05 14:01:40,702:(0.00s):r

2021-05-05 14:01:46,880:(0.00s):geometry:Conducting forward proposal...
2021-05-05 14:01:46,882:(0.00s):geometry:Computing proposal order with NetworkX...
2021-05-05 14:01:46,902:(0.02s):geometry:number of atoms to be placed: 1
2021-05-05 14:01:46,906:(0.00s):geometry:Atom index proposal order is [2190]
2021-05-05 14:01:46,908:(0.00s):geometry:omitted_bonds: []
2021-05-05 14:01:46,911:(0.00s):geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
2021-05-05 14:01:47,229:(0.32s):geometry:creating growth system...
2021-05-05 14:01:47,316:(0.09s):geometry:	creating bond force...
2021-05-05 14:01:47,319:(0.00s):geometry:	there are 27 bonds in reference force.
2021-05-05 14:01:47,322:(0.00s):geometry:	creating angle force...
2021-05-05 14:01:47,325:(0.00s):geometry:	there are 85 angles in reference force.
2021-05-05 14:01:47,329:(0.00s):geometry:	creating torsion force...
2021-05-05 14:01:47,331:(0.00s):geometry:	creating extra

conducting subsequent work with the following platform: CUDA


2021-05-05 14:01:48,098:(0.64s):geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


...

2021-05-05 14:01:55,505:(1.29s):geometry:total reduced potential before atom placement: 48.41899607369698


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


2021-05-05 14:01:57,042:(1.54s):geometry:total reduced energy added from growth system: -5.056836675687361
2021-05-05 14:01:57,046:(0.00s):geometry:final reduced energy 43.36216011253515
2021-05-05 14:01:57,048:(0.00s):geometry:sum of energies: 43.362159398009624
2021-05-05 14:01:57,051:(0.00s):geometry:magnitude of difference in the energies: 7.145255302987152e-07
2021-05-05 14:01:57,053:(0.00s):geometry:Final logp_proposal: 11.562669409973775


added energy components: [('CustomBondForce', 0.0879842954280575), ('CustomAngleForce', 0.04605409749261916), ('CustomTorsionForce', 0.005423562569706604), ('CustomBondForce', -5.1962986311777435)]


2021-05-05 14:01:57,252:(0.20s):root:	writing pickle output...
2021-05-05 14:01:57,526:(0.27s):root:	successfully dumped pickle.
2021-05-05 14:01:57,532:(0.01s):root:	setup is complete.  Writing proposals and positions for each phase to top_prop dict...
2021-05-05 14:01:57,535:(0.00s):root:	writing atom_mapping.png
2021-05-05 14:01:57,635:(0.10s):root:	 steps per move application: 50
2021-05-05 14:01:57,639:(0.00s):root:	trajectory prefix: out
2021-05-05 14:01:57,642:(0.00s):root:	atom selection detected: not water
2021-05-05 14:01:57,645:(0.00s):root:	no nonequilibrium detected.
2021-05-05 14:01:57,647:(0.00s):root:	cataloging HybridTopologyFactories...
2021-05-05 14:01:57,650:(0.00s):root:		phase: solvent:
2021-05-05 14:01:57,653:(0.00s):root:		writing HybridTopologyFactory for phase solvent...
2021-05-05 14:01:57,655:(0.00s):relative:*** Generating RxnHybridTopologyFactory ***
2021-05-05 14:01:57,658:(0.00s):relative:Beginning nonbonded method, total particle, barostat, and exceptio

In [65]:
setup_options

{'protein_pdb': '/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_fixed_nohet.pdb',
 'ligand_file': '/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/perses-0.8.2.dev0-py3.8.egg/perses/data/cdk2-example/CDK2_ligands.sdf',
 'old_ligand_index': 14,
 'new_ligand_index': 15,
 'forcefield_files': ['amber/protein.ff14SB.xml',
  'amber/tip3p_standard.xml',
  'amber/tip3p_HFE_multivalent.xml'],
 'small_molecule_forcefield': 'gaff-2.11',
 'small_molecule_parameters_cache': 'cdk2-cache.json',
 'pressure': 1.0,
 'temperature': 300.0,
 'solvent_padding': 9.0,
 'save_setup_pickle_as': 'fesetup_hbonds.pkl',
 'n_steps_per_move_application': 50,
 'fe_type': 'repex',
 'checkpoint_interval': 250,
 'n_cycles': 2,
 'n_states': 6,
 'n_equilibration_iterations': 0,
 'trajectory_directory': 'output',
 'trajectory_prefix': 'out',
 'atom_selection': 'not water',
 'phases': ['solvent'],
 'protocol-type': 

In [66]:
htf = setup_dict['hybrid_topology_factories']['solvent']

In [67]:
# Save htf
with open("cdk2_solvent_scale_regions.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [68]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent_scale_regions.pickle", "rb") as f:
    htf = pickle.load(f)

In [69]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[44, 2197]

In [70]:
test_torsion_energies(htf, is_solvated=True)

{235: 78, 270: 113, 297: 140, 298: 141, 303: 146}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954756), ('HarmonicAngleForce', 29.199523609209102), ('PeriodicTorsionForce', 15.076747992617829), ('NonbondedForce', 1595.9438694042371), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.330408258666996), ('CustomTorsionForce', 15.07675469305873), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [71]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent_scale_regions.pickle", "rb") as f:
    htf = pickle.load(f)

In [72]:
test_torsion_energies(htf, is_old=False, is_solvated=True, check_scale=True)

{70: 70, 112: 112, 142: 142, 143: 143, 148: 148}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899664), ('HarmonicAngleForce', 29.284045073271738), ('PeriodicTorsionForce', 15.082814158447658), ('NonbondedForce', 1611.7106389845), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.330408258666996), ('CustomTorsionForce', 15.082821925106408), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.330408258666996), ('CustomTorsionForce', 15.07675469305873), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with th

# 8mer in solvent

In [3]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle
from simtk import openmm, unit
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU


In [4]:
# Test 8-mer peptide in solvent
peptide_filename = resource_filename('perses', 'data/8mer-example/4zuh_peptide_capped.pdb')
pdb = app.PDBFile(peptide_filename)
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
# barostat = None
modeller = app.Modeller(pdb.topology, pdb.positions)
 
system_generator = SystemGenerator(forcefields=forcefield_files,
                                   barostat=barostat,
                                   forcefield_kwargs={'removeCMMotion': False,
                                                      'ewaldErrorTolerance': 0.00025,
                                                      'constraints': app.HBonds,
                                                      'hydrogenMass': 4 * unit.amus},
                                   periodic_forcefield_kwargs=None,
                                   small_molecule_forcefield='gaff-2.11',
                                   nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff},
                                   molecules=None,
                                   cache=None)

modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
topology = modeller.getTopology()
positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(topology)



In [5]:
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.annihilation.relative import RepartitionedHybridTopologyFactory
from perses.tests.utils import validate_endstate_energies
import random

aminos = ['ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE','LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL']

ENERGY_THRESHOLD = 1e-6
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT

chain = 'C'
for res in topology.residues():
    if res.id == '2':
        wt_res = res.name
# aminos_updated = [amino for amino in aminos if amino not in [wt_res, 'PRO', 'HIS', 'TRP', 'PHE', 'TYR']]
# mutant = random.choice(aminos_updated)
mutant = "MET"
print(f'Making mutation {wt_res}->{mutant}')

# Create point mutation engine to mutate residue at id 2 to random amino acid
point_mutation_engine = PointMutationEngine(wildtype_topology=topology,
                                            system_generator=system_generator,
                                            chain_id=chain,
                                            max_point_mutants=1,
                                            residues_allowed_to_mutate=['2'],  # the residue ids allowed to mutate
                                            allowed_mutations=[('2', mutant)],
                                            aggregate=True)  # always allow aggregation

# Create topology proposal
topology_proposal = point_mutation_engine.propose(current_system=system, current_topology=topology)

# Create geometry engine
from perses.rjmc.geometry import FFAllAngleGeometryEngine
geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                           use_sterics=False,
                                           n_bond_divisions=100,
                                           n_angle_divisions=180,
                                           n_torsion_divisions=360,
                                           verbose=True,
                                           storage=None,
                                           bond_softening_constant=1.0,
                                           angle_softening_constant=1.0,
                                           neglect_angles=False,
                                           use_14_nonbondeds=True)

# Create geometry proposal
new_positions, logp_proposal = geometry_engine.propose(topology_proposal, positions, beta,
                                                               validate_energy_bookkeeping=True)
logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, positions, beta,
                                            validate_energy_bookkeeping=True)

if not topology_proposal.unique_new_atoms:
    assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
    assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
    vacuum_added_valence_energy = 0.0
else:
    added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

if not topology_proposal.unique_old_atoms:
    assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
    assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
    subtracted_valence_energy = 0.0
else:
    subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


2021-05-12 14:35:19,936:(0.00s):proposal_generator:	Conducting polymer point mutation proposal...


Making mutation ALA->MET


2021-05-12 14:35:20,598:(0.66s):proposal_generator:Using matching_criterion to chose best atom map
2021-05-12 14:35:20,607:(0.01s):proposal_generator:Scaffold has symmetry of 0
2021-05-12 14:35:20,610:(0.00s):proposal_generator:Two molecules are not similar to have a common scaffold
2021-05-12 14:35:20,612:(0.00s):proposal_generator:Proceeding with direct mapping of molecules, but please check atom mapping and the geometry of the ligands.
2021-05-12 14:35:20,617:(0.01s):proposal_generator:len [{13: 8}, {16: 9}, {15: 9}, {14: 9}, {13: 9}, {12: 9}, {11: 9}, {10: 9}, {16: 8}, {15: 8}, {14: 8}, {10: 7}, {12: 8}, {11: 8}, {10: 8}, {16: 7}, {15: 7}, {14: 7}, {13: 7}, {12: 7}, {11: 7}]
2021-05-12 14:35:20,619:(0.00s):proposal_generator:{13: 8}
2021-05-12 14:35:20,621:(0.00s):proposal_generator:{16: 9}
2021-05-12 14:35:20,624:(0.00s):proposal_generator:{15: 9}
2021-05-12 14:35:20,626:(0.00s):proposal_generator:{14: 9}
2021-05-12 14:35:20,628:(0.00s):proposal_generator:{13: 9}
2021-05-12 14:35:

Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU


2021-05-12 14:35:24,122:(0.09s):geometry:	reduced angle potential = 0.24109202446056813.
2021-05-12 14:35:24,213:(0.09s):geometry:	reduced angle potential = 0.10351355073544276.
2021-05-12 14:35:24,300:(0.09s):geometry:	reduced angle potential = 0.7074697455877275.
2021-05-12 14:35:24,401:(0.10s):geometry:	reduced angle potential = 1.7837773591459973.
2021-05-12 14:35:24,503:(0.10s):geometry:	reduced angle potential = 0.18645622300792566.
2021-05-12 14:35:24,603:(0.10s):geometry:	reduced angle potential = 0.23685532101753554.
2021-05-12 14:35:24,705:(0.10s):geometry:	reduced angle potential = 0.00042950963003874666.
2021-05-12 14:35:24,806:(0.10s):geometry:	reduced angle potential = 0.11426431872972385.
2021-05-12 14:35:24,907:(0.10s):geometry:	reduced angle potential = 0.02256081336582398.
2021-05-12 14:35:25,008:(0.10s):geometry:	reduced angle potential = 0.327419646851777.
2021-05-12 14:35:25,194:(0.19s):geometry:	beginning construction of no_nonbonded final system...
2021-05-12 14:

...

conducting subsequent work with the following platform: CPU


2021-05-12 14:35:32,506:(1.37s):geometry:total reduced potential before atom placement: 168.3666171145445


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU


2021-05-12 14:35:32,726:(0.22s):geometry:total reduced energy added from growth system: 19.113357542933382
2021-05-12 14:35:32,729:(0.00s):geometry:final reduced energy 187.47997478600607
2021-05-12 14:35:32,732:(0.00s):geometry:sum of energies: 187.4799746574779
2021-05-12 14:35:32,734:(0.00s):geometry:magnitude of difference in the energies: 1.2852817476982636e-07
2021-05-12 14:35:32,737:(0.00s):geometry:Final logp_proposal: -30541.11542553462


Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU
added energy components: [('CustomBondForce', 0.04562514569649219), ('CustomAngleForce', 0.11963265647716706), ('CustomTorsionForce', 7.457414093494336), ('CustomBondForce', 11.490685647265389)]


In [14]:
from perses.annihilation.relative import RxnHybridTopologyFactory
flatten_exceptions = True
flatten_torsions = True
endstate = None
scale_regions = None
htf = RxnHybridTopologyFactory(topology_proposal=topology_proposal,
                     current_positions=positions,
                     new_positions=new_positions,
                     use_dispersion_correction=False,
                     functions=None,
                     softcore_alpha=None,
                     bond_softening_constant=1.0,
                     angle_softening_constant=1.0,
                     soften_only_new=False,
                     neglected_new_angle_terms=[],
                     neglected_old_angle_terms=[],
                     softcore_LJ_v2=True,
                     softcore_electrostatics=True,
                     softcore_LJ_v2_alpha=0.85,
                     softcore_electrostatics_alpha=0.3,
                     softcore_sigma_Q=1.0,
                     interpolate_old_and_new_14s=flatten_exceptions,
                     omitted_terms=None,
                     flatten_torsions=flatten_torsions,
                     scale_regions=scale_regions)

2021-05-12 14:37:54,058:(140.35s):relative:*** Generating RxnHybridTopologyFactory ***
2021-05-12 14:37:54,063:(0.01s):relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
2021-05-12 14:37:54,284:(0.22s):relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-05-12 14:37:54,287:(0.00s):relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-05-12 14:37:54,290:(0.00s):relative:No unknown forces.
2021-05-12 14:37:54,292:(0.00s):relative:Nonbonded method to be used (i.e. from old system): 4
2021-05-12 14:37:54,294:(0.00s):relative:Adding and mapping old atoms to hybrid system...
2021-05-12 14:37:54,371:(0.08s):relative:Adding and mapping new atoms to hybrid system...
2021-05-12 14:37:54,374:(0.00s):relative:scale_templates: [['nonscale_lambda'], ['nonscale_region

( nonscale_lambda_bonds * nonscale_region ) * (K/2)*(r-length)^2;K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;


In [15]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent.pickle", "wb") as f:
    pickle.dump(htf, f)

In [7]:
# ( nonscale_lambda_bonds * nonscale_region ) * (K/2)*(r-length)^2;

# K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 
# + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;

# length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 
# + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;


### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [16]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [17]:
test_torsion_energies(htf, is_solvated=True)

{432: 39, 433: 40, 434: 41, 451: 58, 452: 59, 453: 60, 454: 61, 455: 62, 456: 63, 459: 66, 460: 67, 461: 68, 462: 69, 463: 70, 464: 71, 465: 72, 466: 73, 467: 74, 468: 75, 469: 76, 470: 77, 481: 88, 482: 89, 483: 90, 484: 91, 485: 92, 486: 93, 487: 94, 488: 95, 489: 96, 490: 97, 491: 98, 492: 99, 493: 100, 494: 101, 495: 102, 496: 103, 497: 104, 498: 105, 499: 106, 500: 107, 501: 108}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
[('HarmonicBondForce', 7.300097488950959), ('HarmonicAngleForce', 12.445683175437441), ('PeriodicTorsionForce', 156.24350834582398), ('NonbondedForce', -10965.130526294286), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CPU
[('CustomBondForce', 8.321999064998987), ('CustomAngleForce', 410.00115528060746), ('CustomTorsionForce', 156.24350834582413), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [18]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)


In [19]:
test_torsion_energies(htf, is_old=False, is_solvated=True)

{39: 39, 40: 40, 41: 41, 58: 58, 59: 59, 60: 60, 63: 63, 64: 64, 65: 65, 76: 76, 77: 77, 78: 78, 79: 79, 80: 80, 81: 81}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
[('HarmonicBondForce', 8.276373919302495), ('HarmonicAngleForce', 409.88152262413047), ('PeriodicTorsionForce', 165.04207540156267), ('NonbondedForce', -2071.707114579371), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CPU
[('CustomBondForce', 8.321999064998987), ('CustomAngleForce', 410.00115528060746), ('CustomTorsionForce', 165.0420754023654), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


### Test one alchemical region, one scale region

In [20]:
import os
from pkg_resources import resource_filename
from perses.app import setup_relative_calculation
import pickle
from simtk import openmm, unit
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import numpy as np

In [21]:
# Test 8-mer peptide in solvent
peptide_filename = resource_filename('perses', 'data/8mer-example/4zuh_peptide_capped.pdb')
pdb = app.PDBFile(peptide_filename)
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
# barostat = None
modeller = app.Modeller(pdb.topology, pdb.positions)
 
system_generator = SystemGenerator(forcefields=forcefield_files,
                                   barostat=barostat,
                                   forcefield_kwargs={'removeCMMotion': False,
                                                      'ewaldErrorTolerance': 0.00025,
                                                      'constraints': app.HBonds,
                                                      'hydrogenMass': 4 * unit.amus},
                                   periodic_forcefield_kwargs=None,
                                   small_molecule_forcefield='gaff-2.11',
                                   nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff},
                                   molecules=None,
                                   cache=None)

modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9*unit.angstroms, ionicStrength=0.15*unit.molar)
topology = modeller.getTopology()
positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
system = system_generator.create_system(topology)



In [22]:
from perses.rjmc.topology_proposal import PointMutationEngine
from perses.annihilation.relative import RepartitionedHybridTopologyFactory
from perses.tests.utils import validate_endstate_energies
import random

aminos = ['ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE','LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL']

ENERGY_THRESHOLD = 1e-6
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT

chain = 'C'
for res in topology.residues():
    if res.id == '2':
        wt_res = res.name
aminos_updated = [amino for amino in aminos if amino not in [wt_res, 'PRO', 'HIS', 'TRP', 'PHE', 'TYR']]
mutant = random.choice(aminos_updated)
print(f'Making mutation {wt_res}->{mutant}')

# Create point mutation engine to mutate residue at id 2 to random amino acid
point_mutation_engine = PointMutationEngine(wildtype_topology=topology,
                                            system_generator=system_generator,
                                            chain_id=chain,
                                            max_point_mutants=1,
                                            residues_allowed_to_mutate=['2'],  # the residue ids allowed to mutate
                                            allowed_mutations=[('2', mutant)],
                                            aggregate=True)  # always allow aggregation

# Create topology proposal
topology_proposal = point_mutation_engine.propose(current_system=system, current_topology=topology)

# Create geometry engine
from perses.rjmc.geometry import FFAllAngleGeometryEngine
geometry_engine = FFAllAngleGeometryEngine(metadata=None,
                                           use_sterics=False,
                                           n_bond_divisions=100,
                                           n_angle_divisions=180,
                                           n_torsion_divisions=360,
                                           verbose=True,
                                           storage=None,
                                           bond_softening_constant=1.0,
                                           angle_softening_constant=1.0,
                                           neglect_angles=False,
                                           use_14_nonbondeds=True)

# Create geometry proposal
new_positions, logp_proposal = geometry_engine.propose(topology_proposal, positions, beta,
                                                               validate_energy_bookkeeping=True)
logp_reverse = geometry_engine.logp_reverse(topology_proposal, new_positions, positions, beta,
                                            validate_energy_bookkeeping=True)

if not topology_proposal.unique_new_atoms:
    assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
    assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
    vacuum_added_valence_energy = 0.0
else:
    added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

if not topology_proposal.unique_old_atoms:
    assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
    assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
    subtracted_valence_energy = 0.0
else:
    subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential


2021-05-12 14:38:15,406:(20.62s):proposal_generator:	Conducting polymer point mutation proposal...


Making mutation ALA->GLN


2021-05-12 14:38:15,910:(0.50s):proposal_generator:Using matching_criterion to chose best atom map
2021-05-12 14:38:15,917:(0.01s):proposal_generator:Scaffold has symmetry of 0
2021-05-12 14:38:15,920:(0.00s):proposal_generator:Two molecules are not similar to have a common scaffold
2021-05-12 14:38:15,922:(0.00s):proposal_generator:Proceeding with direct mapping of molecules, but please check atom mapping and the geometry of the ligands.
2021-05-12 14:38:15,927:(0.00s):proposal_generator:len [{14: 8}, {16: 9}, {15: 9}, {14: 9}, {13: 9}, {12: 9}, {11: 9}, {16: 8}, {15: 8}, {11: 7}, {13: 8}, {12: 8}, {11: 8}, {16: 7}, {15: 7}, {14: 7}, {13: 7}, {12: 7}]
2021-05-12 14:38:15,929:(0.00s):proposal_generator:{14: 8}
2021-05-12 14:38:15,931:(0.00s):proposal_generator:{16: 9}
2021-05-12 14:38:15,933:(0.00s):proposal_generator:{15: 9}
2021-05-12 14:38:15,935:(0.00s):proposal_generator:{14: 9}
2021-05-12 14:38:15,937:(0.00s):proposal_generator:{13: 9}
2021-05-12 14:38:15,939:(0.00s):proposal_gen

Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU


2021-05-12 14:38:19,181:(0.09s):geometry:	reduced angle potential = 0.008352123035916375.
2021-05-12 14:38:19,271:(0.09s):geometry:	reduced angle potential = 0.015784078770126962.
2021-05-12 14:38:19,364:(0.09s):geometry:	reduced angle potential = 0.9898100750114333.
2021-05-12 14:38:19,456:(0.09s):geometry:	reduced angle potential = 0.6134708757443885.
2021-05-12 14:38:19,561:(0.11s):geometry:	reduced angle potential = 0.19548685491694504.
2021-05-12 14:38:19,666:(0.11s):geometry:	reduced angle potential = 1.5697605249689077.
2021-05-12 14:38:19,771:(0.10s):geometry:	reduced angle potential = 0.060108167491294905.
2021-05-12 14:38:19,876:(0.11s):geometry:	reduced angle potential = 0.027996195506081057.
2021-05-12 14:38:19,981:(0.11s):geometry:	reduced angle potential = 0.06875139835281359.
2021-05-12 14:38:20,088:(0.11s):geometry:	reduced angle potential = 3.4001666518231564.
2021-05-12 14:38:20,292:(0.20s):geometry:	beginning construction of no_nonbonded final system...
2021-05-12 14

...

conducting subsequent work with the following platform: CPU


2021-05-12 14:38:26,658:(1.34s):geometry:total reduced potential before atom placement: 168.3666171145445


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU


2021-05-12 14:38:26,904:(0.25s):geometry:total reduced energy added from growth system: 19.113357542933382
2021-05-12 14:38:26,907:(0.00s):geometry:final reduced energy 187.47997478600607
2021-05-12 14:38:26,910:(0.00s):geometry:sum of energies: 187.4799746574779
2021-05-12 14:38:26,912:(0.00s):geometry:magnitude of difference in the energies: 1.2852817476982636e-07
2021-05-12 14:38:26,914:(0.00s):geometry:Final logp_proposal: -30571.76077952775


Desired platform not supported. exception raised: Error initializing CUDA: CUDA_ERROR_NO_DEVICE (100) at /home/conda/feedstock_root/build_artifacts/openmm_1613569706035/work/platforms/cuda/src/CudaContext.cpp:138
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU
added energy components: [('CustomBondForce', 0.04562514569649219), ('CustomAngleForce', 0.11963265647716706), ('CustomTorsionForce', 7.457414093494336), ('CustomBondForce', 11.490685647265389)]


In [23]:
from perses.annihilation.relative import RxnHybridTopologyFactory
flatten_exceptions = True
flatten_torsions = True
endstate = None
scale_regions = [list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])]
htf = RxnHybridTopologyFactory(topology_proposal=topology_proposal,
                     current_positions=positions,
                     new_positions=new_positions,
                     use_dispersion_correction=False,
                     functions=None,
                     softcore_alpha=None,
                     bond_softening_constant=1.0,
                     angle_softening_constant=1.0,
                     soften_only_new=False,
                     neglected_new_angle_terms=[],
                     neglected_old_angle_terms=[],
                     softcore_LJ_v2=True,
                     softcore_electrostatics=True,
                     softcore_LJ_v2_alpha=0.85,
                     softcore_electrostatics_alpha=0.3,
                     softcore_sigma_Q=1.0,
                     interpolate_old_and_new_14s=flatten_exceptions,
                     omitted_terms=None,
                     flatten_torsions=flatten_torsions,
                     scale_regions=scale_regions)

2021-05-12 14:38:26,944:(0.03s):relative:*** Generating RxnHybridTopologyFactory ***
2021-05-12 14:38:26,947:(0.00s):relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
2021-05-12 14:38:27,183:(0.24s):relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-05-12 14:38:27,187:(0.00s):relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
2021-05-12 14:38:27,190:(0.00s):relative:No unknown forces.
2021-05-12 14:38:27,193:(0.00s):relative:Nonbonded method to be used (i.e. from old system): 4
2021-05-12 14:38:27,195:(0.00s):relative:Adding and mapping old atoms to hybrid system...
2021-05-12 14:38:27,273:(0.08s):relative:Adding and mapping new atoms to hybrid system...
2021-05-12 14:38:27,276:(0.00s):relative:scale_templates: [['nonscale_lambda', 'scale_lambda_0', 'i

( nonscale_lambda_bonds * nonscale_region+scale_lambda_0_bonds * scale_region_0+interscale_lambda_0_bonds * interscale_region_0 ) * (K/2)*(r-length)^2;K = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * K1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * K2;length = ( 1 * environment_region+lambda_0_bonds_old * alchemical_region_0 ) * length1 + ( 1 * environment_region+lambda_0_bonds_new * alchemical_region_0 ) * length2;


In [24]:
# Save htf
with open("8mer_solvent_scale_region.pickle", "wb") as f:
    pickle.dump(htf, f)

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [25]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [26]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[24,
 25,
 26,
 21,
 5575,
 5576,
 5577,
 5578,
 5579,
 5580,
 5581,
 5582,
 5583,
 5584,
 5585]

In [27]:
test_torsion_energies(htf, is_solvated=True)

{432: 39, 433: 40, 434: 41, 451: 58, 452: 59, 453: 60, 454: 61, 455: 62, 456: 63, 459: 66, 460: 67, 461: 68, 462: 69, 463: 70, 464: 71, 465: 72, 466: 73, 467: 74, 468: 75, 469: 76, 470: 77, 481: 88, 482: 89, 483: 90, 484: 91, 485: 92, 486: 93, 487: 94, 488: 95, 489: 96, 490: 97, 491: 98, 492: 99, 493: 100, 494: 101, 495: 102, 496: 103, 497: 104, 498: 105, 499: 106, 500: 107, 501: 108, 502: 109, 503: 110, 504: 111, 505: 112, 506: 113, 569: 410, 570: 411}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
[('HarmonicBondForce', 7.300097488950959), ('HarmonicAngleForce', 12.445683175437441), ('PeriodicTorsionForce', 156.24350834582398), ('NonbondedForce', -10647.327508532362), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CPU
[('CustomBondForce', 9.324481660673886), ('CustomAngleForce', 293.77726867877516), ('CustomTorsionForce', 156.24350834582413), ('A

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [28]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [29]:
test_torsion_energies(htf, is_old=False, is_solvated=True, check_scale=True)

{39: 39, 40: 40, 41: 41, 58: 58, 59: 59, 60: 60, 63: 63, 64: 64, 65: 65, 76: 76, 77: 77, 78: 78, 79: 79, 80: 80, 81: 81}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
[('HarmonicBondForce', 9.278856514977395), ('HarmonicAngleForce', 293.6576360222982), ('PeriodicTorsionForce', 177.78722443823037), ('NonbondedForce', 1003360.8969928983), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CPU
[('CustomBondForce', 9.324481660673886), ('CustomAngleForce', 293.77726867877516), ('CustomTorsionForce', 177.78722443903297), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!
conducting subsequent work with the following platform: CPU
[('CustomBondForce', 9.324481660673886), ('CustomAngleForce', 293.77726867877516), ('CustomTorsionForce', 156.24350834582413), ('AndersenThermostat

## Barnase:barstar (old htf, no naked charge fix or counterion fix)

### Test one alchemical region, no scale regions

#### Old system PeroidicTorsionForce vs hybrid system CustomTorsionForce

In [3]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [4]:
test_torsion_energies(htf, is_solvated=True)

{10866: 2139, 10867: 2140, 10868: 2141, 10907: 2180, 10908: 2181, 10909: 2182, 10912: 2185, 10913: 2186, 10914: 2187, 10922: 2195, 10923: 2196, 10924: 2197, 10925: 2198, 10926: 2199, 10927: 2200}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 190.9699139958041), ('HarmonicAngleForce', 797.0922707362615), ('PeriodicTorsionForce', 3856.2346108037013), ('NonbondedForce', -25366.343173297213), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 190.97001343592), ('CustomAngleForce', 983.9501231926239), ('CustomTorsionForce', 3856.234842190186), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!


#### New system HarmonicTorsionForce vs hybrid system CustomTorsionForce

In [5]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [6]:
test_torsion_energies(htf, is_old=False, is_solvated=True)

{2139: 2139, 2140: 2140, 2141: 2141, 2180: 2180, 2181: 2181, 2182: 2182, 2183: 2183, 2184: 2184, 2185: 2185, 2186: 2186, 2187: 2187, 2188: 2188, 2191: 2191, 2192: 2192, 2193: 2193, 2194: 2194, 2195: 2195, 2196: 2196, 2197: 2197, 2198: 2198, 2199: 2199, 2200: 2200, 2201: 2201, 2202: 2202, 2203: 2203, 2204: 2204, 2205: 2205, 2206: 2206, 2214: 2214, 2215: 2215, 2216: 2216, 2217: 2217, 2218: 2218, 2219: 2219, 2220: 2220, 2221: 2221, 2222: 2222, 2223: 2223, 2224: 2224, 2225: 2225, 2226: 2226, 2227: 2227, 2228: 2228, 2229: 2229, 2230: 2230}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 190.55887687012904), ('HarmonicAngleForce', 982.3171572386327), ('PeriodicTorsionForce', 3849.684097413479), ('NonbondedForce', -24973.072364300806), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 190.97001343592), ('Custo

### Test one alchemical region, no scale regions

#### Old system HarmonicTorsionForce vs hybrid system CustomTorsionForce

In [7]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [8]:
test_torsion_energies(htf, is_solvated=True, check_scale=True)

{10866: 2139, 10867: 2140, 10868: 2141, 10907: 2180, 10908: 2181, 10909: 2182, 10912: 2185, 10913: 2186, 10914: 2187, 10922: 2195, 10923: 2196, 10924: 2197, 10925: 2198, 10926: 2199, 10927: 2200}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 190.96991399580065), ('HarmonicAngleForce', 797.0922707362635), ('PeriodicTorsionForce', 3856.2346108037004), ('NonbondedForce', -25367.423468547222), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 191.68441661118993), ('CustomAngleForce', 1002.3374923455682), ('CustomTorsionForce', 3856.234842190186), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom torsion force and standard torsion force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 191.68441661118993), ('CustomAngleForce', 1002

#### New system HarmonicTorsionForce vs hybrid system CustomTorsionForce

In [9]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [10]:
test_torsion_energies(htf, is_old=False, is_solvated=True, check_scale=True)

{2139: 2139, 2140: 2140, 2141: 2141, 2180: 2180, 2181: 2181, 2182: 2182, 2183: 2183, 2184: 2184, 2185: 2185, 2186: 2186, 2187: 2187, 2188: 2188, 2191: 2191, 2192: 2192, 2193: 2193, 2194: 2194, 2195: 2195, 2196: 2196, 2197: 2197, 2198: 2198, 2199: 2199, 2200: 2200, 2201: 2201, 2202: 2202, 2203: 2203, 2204: 2204, 2205: 2205, 2206: 2206, 2214: 2214, 2215: 2215, 2216: 2216, 2217: 2217, 2218: 2218, 2219: 2219, 2220: 2220, 2221: 2221, 2222: 2222, 2223: 2223, 2224: 2224, 2225: 2225, 2226: 2226, 2227: 2227, 2228: 2228, 2229: 2229, 2230: 2230}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 191.27328004539896), ('HarmonicAngleForce', 1000.7045263915769), ('PeriodicTorsionForce', 3849.683473240853), ('NonbondedForce', -24737.764251905406), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 191.68441661118993), ('C

In [ ]:
# TO DO: actually test the scaling of the energies matches